# Process CCD Images
This is the code for CCD image preprocessing, which requires the calibration frames such as bias, dark, flat frames. Each frame set helps eliminate the artificial signals and mitigate the intrinsic patterns and noises in the raw images obtained from CCD camera and optical system. 
  - Read the parameters of WORKDIR, BINNING from **tape.par** (parameter file)
  - Find the images in WORKDIR, having xbinning and ybinning = BINNING in their fits header. 
  - Make the list for image processing. (wbias.list, wdark100s.list, wflatB.list, ... )
  - Generate master BIAS, DARK for each exposure, FLAT for each filter.
  - Do image preprocessing of the object files => w*.fits


Read the parameter file **tape.par**. <br> 
Move to the working directory, WORKDIR. 


In [1]:
import os, time
from glob import glob
import numpy as np 
from astropy.io import fits 
from photlib import read_params, prnlog
import shutil

# READ the parameter file
par = read_params() 

# PRINT the parameters 
WORKDIR = par['WORKDIR']
BINNING = int(par['BINNING'])
prnlog('#WORK: run_ccdproc')
prnlog(f'#WORK DIR: {WORKDIR}')
prnlog(f'#BINNING for processing: {BINNING}')

# MOVE to the working directory =======
CDIR = os.path.abspath(os.path.curdir)
os.chdir(WORKDIR)
#======================================

c:\Users\kang\anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.25.1
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


#WORK: run_ccdproc
#WORK DIR: ./180326-HAT-P-12b
#BINNING for processing: 1


Make the list of FITS files

In [2]:
#============= YOU SHOUD CHECK THE FILE NAMES ================
# MAKE object + bias + dark + flat image file list 
# modify wild cards (*.fits) for your observation file name
biaslist = glob('bias-*.fits') 
darklist = glob('dark-*.fits') 
flatlist = glob('flat-*.fits') 
objlist = glob('object-*.fits')
prnlog(f"#OBJECT IMAGE: {len(objlist)}")
prnlog(f"#BIAS FRAME: {len(biaslist)}")
prnlog(f"#DARK FRAME: {len(darklist)}")
prnlog(f"#FLAT FRAME: {len(flatlist)}")
flist = biaslist + darklist + flatlist + objlist
#============= YOU SHOUD CHECK THE FILE NAMES ================

#OBJECT IMAGE: 44
#BIAS FRAME: 5
#DARK FRAME: 5
#FLAT FRAME: 5


Read the FITS header information from each file and list up the fits files with xbinning and ybinning = BINNING (in **tape.par**)

In [3]:
# DEFINE the list for information of FITS files 
TARGET, TYPE, DATEOBS, EXPTIME, FILTER, FNAME = [],[],[],[],[],[]
for fname in flist: 
    hdu = fits.open(fname)
    img, hdr = hdu[0].data, hdu[0].header
    if hdr.get('XBINNING') != BINNING: continue
    if hdr.get('YBINNING') != BINNING: continue
    # READ the FITS file header and INPUT into the list 
    TARGET.append(hdr['OBJECT'])
    TYPE.append(str.lower(hdr['IMAGETYP']))
    DATEOBS.append(hdr['DATE-OBS'])
    EXPTIME.append(hdr['EXPTIME'])
    FILTER.append(hdr['FILTER'])
    FNAME.append(fname)
# SORT the files for the observation time 
sort_list = np.argsort(DATEOBS) 


Clean up the previous list files in the directory.
 
Generate the file list of each set for image preprocessing 

In [4]:
# LOOP for the all FITS images and 
# GENERATE the file list for each group 
for rname in glob('w*.list'):
    os.remove(rname)
for s in sort_list:
    prnlog(f"{DATEOBS[s]} {TYPE[s]} {FILTER[s]} {EXPTIME[s]} {TARGET[s]}")
    # DEFINE the name of list file with FITS header info.  
    if TYPE[s] == 'bias': 
        lname = 'wbias.list'
    elif TYPE[s] == 'dark': 
        lname = f'wdark{EXPTIME[s]:.0f}s.list'
    elif TYPE[s] == 'flat':
        lname = f'wflat{FILTER[s]}.list'
    elif TYPE[s] == 'object':
        lname = 'wobj.list'
    else:
        lname = 'others.list'
    # WRITE the FITS file name into the LIST file 
    f = open(lname, 'a') 
    f.write(f"{FNAME[s]:s}\n")
    f.close()
    prnlog(f"add {FNAME[s]} to {lname}..." )  


2018-03-25T10:13:55 flat R 15.0 dark180
add flat-046.fits to wflatR.list...
2018-03-25T10:14:20 flat R 15.0 dark180
add flat-047.fits to wflatR.list...
2018-03-25T10:14:47 flat R 17.0 dark180
add flat-048.fits to wflatR.list...
2018-03-25T10:15:15 flat R 19.0 dark180
add flat-049.fits to wflatR.list...
2018-03-25T10:15:46 flat R 19.0 dark180
add flat-050.fits to wflatR.list...
2018-03-26T14:04:06 object R 200.0 hat-p-12b
add object-000115.fits to wobj.list...
2018-03-26T14:07:37 object R 200.0 hat-p-12b
add object-000116.fits to wobj.list...
2018-03-26T14:11:04 object R 200.0 hat-p-12b
add object-000117.fits to wobj.list...
2018-03-26T14:14:30 object R 200.0 hat-p-12b
add object-000118.fits to wobj.list...
2018-03-26T14:24:50 object R 200.0 hat-p-12b
add object-000121.fits to wobj.list...
2018-03-26T14:35:10 object R 200.0 hat-p-12b
add object-000124.fits to wobj.list...
2018-03-26T14:45:30 object R 200.0 hat-p-12b
add object-000127.fits to wobj.list...
2018-03-26T14:55:49 object R 200

Combine the bias frames with median filter, and make the MASTER BIAS (wbias.fits)

In [5]:
# COMBINE bias frames ========================================================
bias_list = np.genfromtxt('wbias.list',dtype='U') 
bias_stack = []
for fname in bias_list:
    hdu = fits.open(fname)[0]
    dat, hdr = hdu.data, hdu.header
    bias_stack.append(dat)
    prnlog(f"{fname:s} {np.mean(dat):8.1f} {np.std(dat):8.1f} {np.max(dat):8.1f} {np.min(dat):8.1f}")
master_bias = np.array(np.median(bias_stack, axis=0), dtype=np.float32)
dat = master_bias
prnlog(f"MASTER BIAS {np.mean(dat):8.1f} {np.std(dat):8.1f} {np.max(dat):8.1f} {np.min(dat):8.1f}")
# WRITE the master bias to FITS
hdr.set('DATE-PRC', time.strftime('%Y-%m-%dT%H:%M:%S'))
hdr.set('OBJECT', 'wbias')
fits.writeto('wbias.fits', master_bias, hdr, overwrite=True)

bias-001.fits    595.4     12.9    800.0    412.0
bias-002.fits    596.3     12.7    795.0    415.0
bias-003.fits    596.6     10.3   2386.0    434.0
bias-004.fits    596.2      9.0   1305.0    430.0
bias-005.fits    595.8      9.0    767.0    434.0
MASTER BIAS    596.1      4.9    648.0    549.0


Combined the dark frames with median filter, and make the MASTER DARK of each exposure. (wdark???s.fits)

In [6]:
# COMBINE dark frames ========================================================
list_files = glob('wdark*.list')
master_darks, exptime_darks = [], [] 
# LOOP of the exposure time
for lname in list_files:
    dark_list = np.genfromtxt(lname, dtype='U') 
    fidx = os.path.splitext(lname)[0]
    dark_stack = [] 
    for fname in dark_list: 
        hdu = fits.open(fname)[0]
        dat, hdr = hdu.data, hdu.header
        dark_stack.append(dat - master_bias) 
        prnlog(f"{fname:s} {np.mean(dat):8.1f} {np.std(dat):8.1f} {np.max(dat):8.1f} {np.min(dat):8.1f}")
    master_dark = np.median(dark_stack, axis=0)
    exptime_dark = hdr.get('EXPTIME')
    dat = master_dark 
    prnlog(f"MASTER DARK {exptime_dark} {np.mean(dat):8.1f} {np.std(dat):8.1f} {np.max(dat):8.1f} {np.min(dat):8.1f}")
    # WRITE the master dark to FITS 
    hdr.set('OBJECT',fidx)
    hdr.set('DATE-PRC', time.strftime('%Y-%m-%dT%H:%M:%S'))
    hdr.set('EXPTIME', exptime_dark)
    fits.writeto(fidx+'.fits', master_dark, hdr, overwrite=True)        
    # MAKE the list of the master dark for each exposure 
    master_darks.append(master_dark)
    exptime_darks.append(exptime_dark)


dark-180s-010.fits    603.2     13.2  11247.0    437.0
dark-180s-011.fits    604.6     12.1   9703.0    442.0
dark-180s-012.fits    603.9     13.1  12944.0    427.0
dark-180s-013.fits    602.7     10.9   6828.0    437.0
dark-180s-014.fits    603.9     12.8   8635.0    438.0
MASTER DARK 180.0      7.6      6.8   1478.0    -49.0


Combine the flat frames with average filter, and make the MASTER FLAT for each filter (wflat??.fits)

In [7]:
# COMBINE the flat frames =======================================================
list_files = glob('wflat*.list') 
master_flats, filter_flats = [], [] 
# LOOP of filters 
for lname in list_files: 
    flat_list = np.genfromtxt(lname, dtype='U') 
    fidx = os.path.splitext(lname)[0]
    flat_stack = [] 
    for fname in flat_list:
        hdu = fits.open(fname)[0]
        dat, hdr = hdu.data, hdu.header
        dat = dat - master_bias
        fdat = dat / np.median(dat)
        flat_stack.append(fdat)
        filter_flat = str.strip(hdr.get('FILTER'))
        prnlog(f"{fname:s} {np.mean(fdat):8.5f} {np.std(fdat):8.5f} {np.max(fdat):8.5f} {np.min(fdat):8.5f}")
    master_flat = np.median(flat_stack, axis=0)
    dat = master_flat
    prnlog(f"MASTER FLAT {filter_flat} {np.mean(dat):8.5f} {np.std(dat):8.5f} {np.max(dat):8.5f} {np.min(dat):8.5f}")
    # WRITE the mater flat to the FITS
    hdr.set('DATE-PRC', time.strftime('%Y-%m-%dT%H:%M:%S'))
    hdr.set('OBJECT', fidx)
    hdr.set('FILTER', filter_flat)
    fits.writeto(fidx+'.fits', master_flat, hdr, overwrite=True)         
    # WRITE the mater flat to the FITS        
    master_flats.append(master_flat)
    filter_flats.append(filter_flat)    

flat-046.fits  0.99967  0.01252  1.36153  0.43132
flat-047.fits  0.99968  0.01265  1.37793  0.42136
flat-048.fits  0.99970  0.01265  1.42655  0.43017
flat-049.fits  0.99971  0.01262  1.49473  0.43088
flat-050.fits  0.99971  0.01299  1.72597  0.43138
MASTER FLAT R  0.99966  0.01129  1.11732  0.43088


Do the preprocessing for all of object images with the calibration frames such as master bias, master dark, master flat. 
Write the result to the FITS (wobject*.fits)

In [8]:
# DO PREPROCESSING with the calibration frames 
flist = np.genfromtxt('wobj.list', dtype='U')
for fname in flist:
    hdu = fits.open(fname)[0]
    cIMAGE, hdr = hdu.data, hdu.header
    cFILTER = str.strip(hdr.get('FILTER'))
    cEXPTIME = float(hdr.get('EXPTIME'))
    # FIND the master dark with the closest exposure time
    dd = np.argmin(np.abs(np.array(exptime_darks) - cEXPTIME))
    dEXPTIME = exptime_darks[dd]
    dFRAME = master_darks[dd]
    dFRAME = dFRAME * (cEXPTIME/dEXPTIME)
    # FIND the master flat with the same filter 
    ff = filter_flats.index(cFILTER)
    fFILTER = filter_flats[ff]
    fFRAME = master_flats[ff]
    cIMAGE = cIMAGE - master_bias - dFRAME
    cIMAGE = np.array(cIMAGE / fFRAME, dtype=np.float32)
    # WRITE the calibrated image to the FITS
    hdr.set('DATE-PRC', time.strftime('%Y-%m-%dT%H:%M:%S'))
    fits.writeto('w'+fname, cIMAGE, hdr, overwrite=True)
    prnlog(f'{fname}({cFILTER},{cEXPTIME})<<[{fFILTER},{dEXPTIME}]')

object-000115.fits(R,200.0)<<[R,180.0]
object-000116.fits(R,200.0)<<[R,180.0]
object-000117.fits(R,200.0)<<[R,180.0]
object-000118.fits(R,200.0)<<[R,180.0]
object-000121.fits(R,200.0)<<[R,180.0]
object-000124.fits(R,200.0)<<[R,180.0]
object-000127.fits(R,200.0)<<[R,180.0]
object-000130.fits(R,200.0)<<[R,180.0]
object-000131.fits(R,200.0)<<[R,180.0]
object-000132.fits(R,200.0)<<[R,180.0]
object-000133.fits(R,200.0)<<[R,180.0]
object-000134.fits(R,200.0)<<[R,180.0]
object-000135.fits(R,200.0)<<[R,180.0]
object-000136.fits(R,200.0)<<[R,180.0]
object-000137.fits(R,200.0)<<[R,180.0]
object-000138.fits(R,200.0)<<[R,180.0]
object-000139.fits(R,200.0)<<[R,180.0]
object-000141.fits(R,200.0)<<[R,180.0]
object-000143.fits(R,200.0)<<[R,180.0]
object-000145.fits(R,200.0)<<[R,180.0]
object-000147.fits(R,200.0)<<[R,180.0]
object-000149.fits(R,200.0)<<[R,180.0]
object-000151.fits(R,200.0)<<[R,180.0]
object-000153.fits(R,200.0)<<[R,180.0]
object-000155.fits(R,200.0)<<[R,180.0]
object-000157.fits(R,200.

Come back to the home directory. 

In [9]:
# RETURN to the directory ===========
os.chdir(CDIR) 
#====================================